# Astropy

In [ ]:
# As usual, we start with some imports
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

## I. Viewing and manipulating FITS images

Originally, reading and manupulating fits file was included in the package `pyfits` developped by stsci. You may still find that this is the way to do when skimming the internet. This is surely not a bad package to use (most of the early astronomical python development come from stsci ) ... but `astropy` includes that facility as well within a larger growing  astronomical working environment. Hence, we will look here how to do within `astropy.io.fits`. The commands with `pyfits` are essentially the same. 

``astropy.io.fits`` provides a lot of flexibility for reading FITS  files and headers, but most of the time the convenience functions are the easiest way to access the data. 

In [ ]:
# The first step is to import fits
from astropy.io import fits

### I.1 Opening FITS files, loading the image data, exploring the header


``fits.getdata()`` reads just the  data from a FITS file, but with the `header=True` keyword argument will also read the header. 

In [ ]:
image_file = 'HorseHead.fits'
get_data, get_header = fits.getdata(image_file, header=True)

You can also add an 'ext' parameter which specifies the extension of the fits file you are interested in: 
i.e. This can be the extension number:
```
        getdata('in.fits', 0)      # the primary header
        getdata('in.fits', 2)      # the second extension
        getdata('in.fits', ext=2)  # the second extension
```
But the can also be the ``EXTNAME`` value (if unique) (e.g. for HST data):
```
        getdata('in.fits', 'sci')
        getdata('in.fits', extname='sci')  # equivalent
```

For those unfamiliar with FITS headers, they consist of a list of 80 byte `cards`, where a card contains a keyword, a value, and a comment. The keyword and comment must both be strings, whereas the value can be a string or an integer, floating point number, complex number, or True/False. Keywords are usually unique within a header, except in a few special cases.

We can also open the fits and see what it contains. The function `fits.open()` is a higher level function that allows one to perform more advanced operations on a fits file. 
You'll see that the real structure can be a bit more complicated than just a header and a data set in binary format. 

In [ ]:
hdu_list = fits.open(image_file)
hdu_list.info()

The fits is made of "HDU" or "Header Data Units". The function `open()` provides a list of HDU objects.  An HDU (Header Data Unit) is the highest level component of the FITS file structure, consisting of a header and (typically) a data array or table. After the above open call, `hdulist[0]` is the primary HDU, `hdulist[1]` is the first extension HDU, etc (if there are any extensions), and so on.

**NOTE:** The open() function will seamlessly open FITS files that have been compressed with gzip, bzip2 or pkzip.

Generally the image information is located in the `PRIMARY` block. The blocks are numbered and can be accessed by indexing `hdu_list`. The scientific data are *not always* in the primary block. Similarly, all the keywords are not stored in the primary HDU. So, remember that if you do not find a keyword of interest in the header (especially while doing `getdata()`), this could simply be because it is not in the pimary header. 

Each element of an HDUList is an HDU object with `.header` and `.data` attributes, which can be used to access the header and data portions of the HDU.

In [ ]:
image_data = hdu_list[0].data

In [ ]:
header_data = hdu_list[0].header

Your data are now stored as a 2-D numpy array.  Want to know the dimensions of the image?  Just look at the `shape` of the array.

In [ ]:
print(type(image_data))
print(image_data.shape)

While your header is saved in a Header object (that has basically the same structure as a dictionary)

In [ ]:
print type(header_data)

**Note**: You can also access the extension by its extension name (specified in the EXTNAME keyword) instead of its ID in the list, i.e. `hdu_list['Primary'].data`
If there is more than one extension with the same `EXTNAME`, the `EXTVER` value needs to be specified along with the `EXTNAME` as a tuple; e.g.: `scidata = hdulist['sci',2].data`

At this point, we can just close the FITS file.  We have stored everything we wanted to a variable.

In [ ]:
hdu_list.close()

When you close(), the header is still available ... but not always the data. 



**Note:** 

If you don't need to examine the FITS header, you can call `fits.getdata` to bypass the previous steps.

### I.2 Viewing the image data and getting basic statistics

In [ ]:
plt.imshow(image_data, cmap='gray')
plt.colorbar()

# To see more color maps
# http://wiki.scipy.org/Cookbook/Matplotlib/Show_colormaps
# See also https://jakevdp.github.io/blog/2014/10/16/how-bad-is-your-colormap/

You can get some basic statistics about your image 

In [ ]:
print 'Min:', np.min(image_data) 
print 'Max:', np.max(image_data) 
print 'Mean:', np.mean(image_data) 
print 'Stdev:', np.std(image_data) 

You can also plot hisograms of some regions of the image. 
To make a histogram with `matplotlib.pyplot.hist()`, you need to cast the data from a 2-D to array to something one dimensional.    

For that purpose, you can use e.g. the iterable python object `img_data.flat`.

In [ ]:
print(type(image_data.flat))
NBINS = 1000
histogram = plt.hist(image_data.flat, NBINS)

### I.3 Displaying the image with a logarithmic scale

Want to use a logarithmic color scale? To do so we need to load the `LogNorm` object from `matplotlib`.

In [ ]:
from matplotlib.colors import LogNorm
plt.imshow(image_data, cmap='gray', norm=LogNorm())

# I chose the tick marks based on the histogram above
cbar = plt.colorbar(ticks=[5.e3,1.e4,2.e4])
cbar.ax.set_yticklabels(['5,000','10,000','20,000'])

### I.4 Basic image math: image stacking

You can perform math with the image data like any other numpy array.  In this particular example, I will stack several images of M13 taken with a ~10'' telescope.

Here, we'll open a series of FITS files and store the data in a list, which will be named `image_concat`.


In [ ]:
image_list = ['M13_blue_000'+n+'.fits' for n in ['1','2','3','4','5'] ]
# The short way  (remember that it is better to use list comprehension)
image_concat = [ fits.getdata(image) for image in image_list]

# The long way
#image_concat = []
#for image in image_list:
#    image_concat.append(fits.getdata(image))

Now I'll stack the images by summing my concatenated list and then visualize the histogram to decide on the best stretch for vizualisation.

In [ ]:
final_image = np.sum(image_concat, axis=0)
image_hist = plt.hist(final_image.flat, 1000)

In [ ]:
plt.imshow(final_image, cmap='gray', vmin=2.e3, vmax=3.e3)
plt.colorbar()

### I.5 Writing image data to a FITS file

This is easy to do with the `writeto()` method.
You will receive an error if the file you are trying to write already exists. To avoid this/allow for overwritting, you should set the option `clobber=True`.

In [ ]:
outfile = 'stacked_M13_blue.fits'

hdu = fits.PrimaryHDU(final_image)
hdu.writeto(outfile, clobber=True)

## II. Edit a FITS header

We will now see how to edit a FITS header, and then write it back out to disk. For this example we're going to change the `OBJECT` keyword.


As we have seen before, there is basically three equivalent ways to access the header: 

1. Through the hdulist: `header = hdulist[0].header`  (0 if you want the header associated with the first HDU)

2. Using the convenience function getdata, specifying the option `header=True`, and also `ext=0`: `data, header = fits.getdata('input_file.fits', header=True, ext=0)`

3. Using a convenience function `fits.getheader()`, specifying the HDU extension:
`header = fits.getheader('input_file.fits', 0)`

Those functions/attribute return a Header instance, another Astropy object. To get the value associated with a header keyword, you do as you do to access elements of a python dictionary:
`header['OBJECT']`. 

In [ ]:
data, header = fits.getdata('stacked_M13_blue.fits', ext=0, header=True)
header

Although keyword names are always in upper case inside the FITS file, specifying a keyword name with Astropy is case-insensitive, for the user’s convenience.

This header is filled only with default values, we can add some cards and e.g. add the correct object name. 

You could do:
'''
header['OBJECT'] = 'M31'
'''
But as the header does not exist yet, you do not have any description of that keyword. If yo want to also add a description you may need to set a tuple value: 

In [ ]:
header['OBJECT'] = ('M31', 'Object name')
header

Finally, we have to write out the FITS file. Again, the convenience  function for this is the most useful command to remember:

In [ ]:
fits.writeto('output_file.fits', data, header, clobber=True)

The option `clobber = True` is needed to overwrite the file if already existing. 

If insetead of using getdata, you have worked only with hdulist (and modified `data` and/or `header` instances), you can also save your new fits (with all its layers !) using: 
`hdulist.writeto('newfile.fits')`

In [ ]:
hdu_list = fits.open(image_file)
hdu_list[0].data = hdu_list[0].data*1.5  # I perform a simple operation on the "data" of the first HDU
hdu_list.writeto('new_file.fits', clobber=True)
hdu_list.close()

### II.2 Creating a brand-new fits image file:

Imagine you have an array containing your data (e.g. a model PSF, a model sky, ...) that you want to save as a fits file. For simplicity here, we'll consider an arbitrary array:

In [ ]:
myarray = np.ones(shape=(20, 10))
hdu = fits.PrimaryHDU(myarray)

We then create a HDUList to contain the newly created primary HDU, and write to a new file:

In [ ]:
hdulist = fits.HDUList([hdu])
hdulist.writeto('new.fits')

In fact, Astropy even provides a shortcut for the last two lines to accomplish the same behavior: `hdu.writeto('new.fits')`

### II.3 `HISTORY` and `COMMENT` cards:

Some of the fits CARDS have different properties than the others. Normal cards are unique and should be no larger than 80 characters. 
For those cards, each new entry will be appended after the previous one (i.e. those keywords can be seen as lists on multiple elements, while other keywords only have one element. 

In [ ]:
header['COMMENT'] = 'Night was not photometric' 

# If you have "datetime"package installed, you may automatically add the date to the HISTORY card
try:
    from datetime import datetime
    now = datetime.datetime.now()
    header['HISTORY'] = 'I updated this file on'+now.strftime('%Y-%m-%d %H:%M:%S')
except:
    now = '2017-05-15'
    header['HISTORY'] = 'I updated this file on'+now.strftime('%Y-%m-%d %H:%M:%S')
header

In [ ]:
header['COMMENT'] = 'Combination of 10 images'
header['COMMENT'][2] = ''
print header['COMMENT']

In [ ]:
header['COMMENT'][2] = 'Combination of 5 images'
print header['COMMENT']

### Exercise: 

Read in the file `stacked_M13_blue.fits` we wrote above, and add three header keywords/cards:

1. 'RA' for the Right Ascension of M31
2. 'DEC' for the Declination of M31
3. 'COMMENT' where you specify that you added RA,DEC manually and give its source

then write the updated header back out to a new file. 

Right Ascension and Declination of M31:    
(RA, DEC) = ('16h41m41.44s', '36d27m36.9s')

### III. Fits Tables

We can proceed as for images to see the content of the file.    
Here, we will use a big fits table corresponding to a Chandra X-ray observation "chandra_events.fits". Since the file is big, we will open with `memmap=True` to prevent RAM storage issues.

In [ ]:
event_filename = 'chandra_events.fits'
hdu_list = fits.open(event_filename, memmap=True)
hdu_list.info()

We are interested in reading `EVENTS`, which contains information about each X-ray photon that hit the detector. 

We see in the `Dimensions` columns of the output of `info` that the different HDU (from the second one) contains a certain number of "rows" (`R`) and "columns" (`C`). You can print the column name simply using the attribute `columns` : 

In [ ]:
hdu_list[1].columns

To access the table data (content of the columns) you can follow the same procedure as for images, with the possibility to select a specific column:

In [ ]:
event_data = hdu_list[1].data

In [ ]:
type(event_data)

I can now for example plot an histogram of the events energy:

In [ ]:
he = plt.hist(event_data['energy'], bins=500)
plt.show()

In [ ]:
# We ca now close the fits file
hdu_list.close()

## III. Image coordinates

### III.1 Overlaying image coordinates

To overlay coordinates on an image, we need to import the `WCS` module of `astropy.wcs`.

In [ ]:
from astropy.wcs import WCS

In [ ]:
image_file = 'HorseHead.fits'
data, header = fits.getdata(image_file, header=True)

We can check in the header that information regarding coordinates exist in there. We have seen below that the list of cards can sometimes be very long and that it can be tedious to find the card of interest. Fortunately, we can filter a bit using `*` to replace characters. Things related to "object" will start with an `O` ... so let's filter this way: 

In [ ]:
header['o*']

Yeah ! Good guess, we see the object RA, DEC are there. But this is not enough, we need to know the complete coordinate transformation system.  
Now, let's check that the coordinate system and transformation (pixels -> physical coordinates) is also defined in the header. 

This information is generally located in cards starting with "C": `CTYPE`, `CRPIX`, `CRVAL`, ... :

In [ ]:
header['c*']

In [ ]:
wcs.get_axis_types

In [ ]:
wcs = WCS(header)
ax = plt.subplot(projection=wcs)   # This initialises your axes transforming it to WCS 
ax.imshow(data, origin='lower')
ax.grid(color='white', ls='solid')
ax.set_xlabel('Right Ascension (J2000)')
ax.set_ylabel('Declination (J2000)')

You can overlay a different system of axis coordinates easily

In [ ]:
wcs = WCS(header)
ax = plt.subplot(projection=wcs)   # This initialises your axes transforming it to WCS 
ax.imshow(data, origin='lower')
ax.grid(color='white', ls='solid')
ax.set_xlabel('Right Ascension (J2000)')
ax.set_ylabel('Declination (J2000)')
overlay = ax.get_coords_overlay('galactic')
overlay.grid(color='white', ls='dotted')
overlay[0].set_axislabel('Galactic longitude')
overlay[1].set_axislabel('Galactic latitude')

You may want to refine a bit the labeling of the axis (let's drop here the galactic coordinates for clarity)

In [ ]:
wcs = WCS(header)
ax = plt.subplot(projection=wcs)
ax.imshow(data, origin='lower')
ax.grid(color='white', ls='solid')
ax.set_xlabel('Right Ascension (J2000)')
ax.set_ylabel('Declination (J2000)')

cRA = ax.coords[0]
cDEC = ax.coords[1]
cRA.set_major_formatter('hh:mm:ss')
cDEC.set_major_formatter('dd:mm:ss')
plt.show()

For more options regarding fine tuning of your grid overlay, you may consult: http://docs.astropy.org/en/stable/visualization/wcsaxes/ticks_labels_grid.html . 

Let's note that there is another powerful (astropy-affiliated) package (developped by Robitaille and Bressert) that allows coordinate overlays with a lot of functionalities, checking for you the coordinate system and equinox, and all that with a somehow more simple syntax (but new objects): `aplpy`.   

In [ ]:
# !pip install aplpy  # If aplpy is not installed
# Have a look to https://aplpy.github.io/
import aplpy
fig = plt.figure()
img = aplpy.FITSFigure(image_file, figure=fig)
img.show_grayscale(stretch='arcsinh', invert=True)
img.tick_labels.set_xformat('hh:mm:ss')
img.tick_labels.set_yformat('dd:mm:ss')
img.add_grid()
img.set_grid_color('black')
img.set_grid_alpha(0.5)

### III.2 Basic operations with coordinates

To use those coordinates and calculate e.g. separations between objects, we need to import `SkyCoord` module from `astropy.coordinates`. In addition, as "coordinates" always have units attached, we need to import `astropy.units` that will allow us to work with "unit-attached quantities".

In [ ]:
from astropy.coordinates import SkyCoord
from astropy import units as u

In [ ]:
# Coordinates can be instanciated in various ways
# E.g. giving the coordinate in degree
c = SkyCoord(10.625, 41.2, frame='icrs', unit='deg')
c

But you can also explicitly attach the units to your variable using: `u.degree`

In [ ]:
RA, DEC = 10.625*u.degree, 41.2*u.degree
c = SkyCoord(ra=RA, dec=DEC, frame='icrs')
print '(RA, DEC) = ', RA, DEC
print 'c', c

I always have hard time playing with units in degree ... I prefer `hh:mm:ss` / `dd:mm:ss`. No problem:

In [ ]:
c = SkyCoord('00h42.5m', '+41d12m')
c

You can specify explictly that your coordinates is effectively ICRS ([International Celestial Reference System](https://en.wikipedia.org/wiki/International_Celestial_Reference_System)) -i.e. this is almost Equatorial J2000- using the optional argument `frame`. If not, you can specify the system through the argument `frame` (e.g. galactic, FK5, ...). 

Note that if you give RA, DEC and specify `frame=galactic`, RA and DEC will be converted and galactic coordinates will be stored. You need to specify arguments `l=my_galactic_longitude, b=my_galactic_latitude` to get the coordinates read as a longitude/latitude. 

In [ ]:
c = SkyCoord('00h42m30s', '+41d12m00s', frame='icrs')
print c

Another possibility is to do the following:

In [ ]:
c = SkyCoord('00 42 30 +41 12 00', unit=(u.hourangle, u.deg))
c = SkyCoord('00:42.5 +41:12', unit=(u.hourangle, u.deg))
c

In [ ]:
c = SkyCoord('5h40m50s', '-2d27m26s')
c

Once you have a coordinate object you can now access the components of that coordinate (e.g. RA, Dec) and get a specific string representation of the full coordinate.

In [ ]:
c.ra 

In [ ]:
c.ra.hour  

In [ ]:
c.ra.hms  

In [ ]:
c.dec.radian

Coordinates can easily be converted to strings using the `to_string()` method:

In [ ]:
print 'c.to_string(\'decimal\') ->', c.to_string('decimal')
print 'c.to_string(\'dms\')     ->', c.to_string('dms')
print 'c.to_string(\'hmsdms\')  ->', c.to_string('hmsdms')

And if you want to know your coordinate in another coordinate system:

In [ ]:
print c.galactic 
print 'A more flexible way is to use transform_to:'
print c.transform_to('galactic')

Transform to 'FK5' and change to get coordinates for an old J1950 equinox. 

In [ ]:
from astropy.coordinates import FK5
c_fk5 = c.transform_to('fk5')
print 'coord J2000:', c_fk5
c_J1950 = c_fk5.transform_to(FK5(equinox='J1950'))  # precess to a different equinox 
print 'coord, J1950', c_J1950

### III.3 Coordinates and images

Let's now see how to overlay a symbol on an image at a specific position.    
Let's work with the image of the Horse Head nebula again. Remember that we have done:
```
data, header = fits.getdata(image_file, header=True)
wcs = WCS(header)
```
To avoid confusion with `wcs` package, let's rename our variable `wcs -> w`

In [ ]:
# To avoid confusion with wcs package, let's rename our variable wcs -> w
w = wcs.copy()
# Let's look to some pixel coordinates of interest
pixcrd = np.array([[500, 300], [224, 438]], np.float_)

# Convert pixel coordinates to world coordinates
# The second argument is "origin" -- in this case we're declaring we
# have 1-based (Fortran-like) coordinates.
world = w.wcs_pix2world(pixcrd, 1)
print(world)

You can now convert those coordinates into `SkyCoord` instances:

In [ ]:
world_SC = [SkyCoord(wc[0], wc[1], unit='deg') for wc in world]
world_SC

You can also measure the separation between those two pixels ... 

In [ ]:
world_SC[0].separation(world_SC[1])

... and convert this into arcseconds. 
The separation is an angle and you can convert it to other angular units using the method `to(newunits)`:

In [ ]:
world_SC[0].separation(world_SC[1]).to(u.arcsec)

Now you can also overlay those to pixels on your image (but you simply have to give their pixel coordinates ... nothing fancy in there !). Conversely, you may want to see to which pixel corresponds a specific coordinate. You therefore need to use a `wcs.WCS` method to convert a coordinate into pixel.  

In [ ]:
w.wcs_world2pix( world_SC[0].ra.value, world_SC[0].dec.value, 1 )  
# Again 1 is here to fix the origin of the coordinate system 

Note that above, we have used taken the "unitless" value of the `RA` and `DEC`. For this purpose, we have used the `units` method `value` to extract only the numerical value of `RA`/`DEC` as given by `world_SC[0].ra`/`world_SC[0].dec`.   

In [ ]:
print world_SC[0].ra.value, world_SC[0].dec.value
print 85.2598025546, -2.49940926988

In [ ]:
wcs = WCS(header)
ax = plt.subplot(projection=w)
ax.imshow(data, origin='lower')
ax.grid(color='white', ls='solid')
ax.set_xlabel('Right Ascension (J2000)')
ax.set_ylabel('Declination (J2000)')

mrk = ['x', 'd']
[ax.plot(px[0], px[1], marker=mrk[i], color='red') for i, px in enumerate(pixcrd)]


newpx = w.wcs_world2pix(85.2296, -2.4434, 1)
print newpx
ax.plot(newpx[0], newpx[1], marker='o', color='orange')

## IV. Use of "Quantities" for astrophysical calculation

Astropy's `Quantity` object can make astrophysics calculations easier. The example include calculating the mass of a galaxy from its velocity dispersion.  

Using `Quantity` object could also be a good practice for using quantities in functions you distribute to other people. This may avoid people assuming different units and making big mistakes !

As we have seen above, attaching units to variable is done using `astropy.units`. It is conventional to import it as `u`, so your import should look like `import astropy.units as u`. 

Astropy also has a `constants` module, where typical physical constants are available.  The constants are stored as objects of a subclass of `Quantity`, so they behave just like a `Quantity`. Here, we'll only need the gravitational constant `G`, Planck's constant `h`, and Boltzmann's constant, `k_B`.

In [ ]:
#import astropy.units as u
from astropy.constants import G, h, k_B

In this example, we will use `Quantity` objects to estimate a hypothetical galaxy's mass, given its half-light radius and radial velocities of stars in the galaxy.

Lets assume that we measured the half light radius of the galaxy  to be 29 pc projected on the sky at the distance of the galaxy.  This radius is often called the "effective radius", so we will store it as a `Quantity` object with the name `Reff`. The easiest way to create a `Quantity` object is just by multiplying the value with its unit. Units are accessed as u."unit", in this case u.pc.

In [ ]:
Reff = 29 * u.pc

A completely equivalent (but more verbose) way of doing the same thing is to use the `Quantity` object's initializer, demonstrated below.  In general, the simpler form (above) is preferred, as it is closer to how such a quantity would actually be written in text.  The initalizer form has more options, though, which you can learn about from the [astropy reference documentation on Quantity](http://docs.astropy.org/en/stable/api/astropy.units.quantity.Quantity.html).

In [ ]:
Reff = u.Quantity(29, unit=u.pc)

We can access the value and unit of a `Quantity` using the `value` and `unit` attributes.

In [ ]:
print 'Half light radius %.2f %s' %(Reff.value, Reff.unit)

Next, we will first create a synthetic dataset of radial velocity measurements, assuming a normal distribution with a mean velocity of 206 km/s and a velocity dispersion of 250 km/s.

In [ ]:
vmean = 206
sigin = 4.6
v = np.random.normal(vmean, sigin, 500)*u.km/u.s
# For the illustration we convert to m/s
print 'First 10 radial velocity measurements :' 
print ['%.2f %s ' %(v.to(u.m/u.s)[i].value, v.to(u.m/u.s)[i].unit) for i in range(10)]


In [ ]:
plt.figure()
plt.hist(v, bins=20, histtype="step")
plt.xlabel("Velocity (km/s)")
plt.ylabel("N")

Next, we calculate the velocity dispersion of the galaxy.  This demonstrates how you can perform basic operations like subtraction and division with `Quantity` objects, and also use them in standard numpy functions such as `mean()` and `size()`. They retain their units through these operations just as you would expect them to.

In [ ]:
sigma = np.sqrt(np.sum((v - np.mean(v))**2) / np.size(v))
print 'Velocity dispersion: %.2f %s' %(sigma.value, sigma.unit)

**Note**: In general, you should only use `numpy` functions with `Quantity` objects, *not* the `math` equivalents, unless you are sure you understand the consequences.

Now for the actual mass calculation.  If a galaxy is pressure-supported (for example, an elliptical or dwarf spheroidal galaxy), its mass within the stellar extent can be estimated using a straightforward formula: 

$$
M_{1/2}=\frac{4\sigma^2 R_{eff}}{G}.
$$  

There are caveats to the use of this formula for science - see Wolf et al. 2010 for details.  For demonstrating `Quantity`, just accept that this is often good enough. For the calculation we can just multiply the quantities together, and `astropy` will keep track of the units.

In [ ]:
M = 4*sigma**2*Reff/G
M

The result is in a composite unit, so it's not really obvious it's a mass. However, it can be decomposed to cancel all of the length units ($\rm{km}^2 \rm{pc}/\rm{m}^3$) using the `decompose()` method.

In [ ]:
M.decompose()

We can also easily express the mass in whatever form you like - solar masses are common in astronomy, or maybe you want the default SI and CGS units.

In [ ]:
print 'Galaxy mass'
print 'in solar units: %.3e %s' %(M.to(u.Msun).value, M.to(u.Msun).unit) 
print 'SI units: %.3e %s' %(M.si.value, M.si.unit)
print 'CGS units: %.3e %s' %(M.cgs.value, M.cgs.unit)

Or, if you want the log of the mass, you can just use ``np.log10`` as long as the logarithm's argument is dimensionless.

In [ ]:
np.log10(M / u.Msun)

However, you can't take the log of something with units, as that is not mathematically sensible.

In [ ]:
np.log10(M)

#### Exercise:

Use `Quantity` and Kepler's law in the form given below to determine the (circular) orbital speed of the Earth around the sun in km/s. You should not have to look up an constants or conversion factors to do this calculation - it's all in `astropy.units` and `astropy.constants`.

$$v = \sqrt{\frac{G M_{\odot}}{r}}$$

(Completely optional, but a good way to convince yourself of the value of Quantity:) Do the above calculations by hand - you can use a calculator (or python just for its arithmatic) but look up all the appropriate conversion factors and use paper-and-pencil approaches for keeping track of them all.  Which one took longer?

## XX References

This notebook is mostly based on the astropy tutorials available here: http://www.astropy.org/astropy-tutorials/

- Your reference for using astropy should be the online documentation http://docs.astropy.org/en/latest

- Documentation regarding the wcs module: http://docs.astropy.org/en/stable/visualization/wcsaxes/

- Documentation regarding the use of coordinates: http://docs.astropy.org/en/stable/coordinates/index.html

- The doc of astropy.io.fits also provides relevant information: http://docs.astropy.org/en/stable/io/fits/#f1

- For an in-depth discussion of `Quantity` objects, see the [astropy documentation section](http://docs.astropy.org/en/stable/units/quantity.html). See also http://docs.astropy.org/en/stable/units/ for various informations of interest regarding the use of units in general !

- How bad is your color map ? (aka how not to be fooled by a poor choice of color map): https://jakevdp.github.io/blog/2014/10/16/how-bad-is-your-colormap/